In [17]:
import lightgbm as lgb
import pandas as pd

# Notes:
Deal with possibility for Going changing over day - generate with all types?

In [18]:

df = pd.read_csv("full_dataset.csv")

df

/Users/hugowickham/Desktop/horseracing/.venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (25,35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,Date of Race,Country,Track,Going,Type,Distance,Class,Time,Stall,...,Going Wins,Last time out Position,Course Winner,Distance Winner,Up in Trip,Betfair Place SP,Placed in Betfair Market,Place Return,Place Lay Return,Betfair Rank
0,0,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,NaN,...,NaN,,,,,NaN,,0.00,0.0,NaN
1,1,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,NaN,...,NaN,,,,,NaN,,0.00,0.0,NaN
2,2,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,NaN,...,NaN,,,,,NaN,,0.00,0.0,NaN
3,3,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,NaN,...,NaN,,,,,NaN,,0.00,0.0,NaN
4,4,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,NaN,...,NaN,,,,,NaN,,0.00,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838950,51746,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,17:25:00,14.0,...,0.0,11,NO,YES,NO,3.97,NO,-10.00,9.8,6.0
2838951,51747,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,17:25:00,7.0,...,0.0,2,NO,NO,YES,5.50,NO,-10.00,9.8,7.0
2838952,51748,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,17:25:00,6.0,...,2.0,5,NO,YES,NO,6.80,YES,56.84,-58.0,9.0
2838953,51749,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,17:25:00,8.0,...,0.0,3,NO,NO,NO,6.80,YES,56.84,-58.0,8.0


# Race id

In [19]:
df["Time"] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time
df = df.rename(columns={"Date of Race":"date"})
df["rid"] = df["date"].astype(str).str.replace("-","") + df["Track"] + df["Time"].apply(lambda x: f"{x.hour:02d}{x.minute:02d}")

cols = ['rid'] + [col for col in df.columns if col != 'rid']
df = df[cols]

df

,rid,Unnamed: 0,date,Country,Track,Going,Type,Distance,Class,Time,...,Going Wins,Last time out Position,Course Winner,Distance Winner,Up in Trip,Betfair Place SP,Placed in Betfair Market,Place Return,Place Lay Return,Betfair Rank
0,20020406AINTREE1345,0,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,...,NaN,,,,,NaN,,0.00,0.0,NaN
1,20020406AINTREE1345,1,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,...,NaN,,,,,NaN,,0.00,0.0,NaN
2,20020406AINTREE1345,2,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,...,NaN,,,,,NaN,,0.00,0.0,NaN
3,20020406AINTREE1345,3,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,...,NaN,,,,,NaN,,0.00,0.0,NaN
4,20020406AINTREE1345,4,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,...,NaN,,,,,NaN,,0.00,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838950,20250514YORK1725,51746,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,17:25:00,...,0.0,11,NO,YES,NO,3.97,NO,-10.00,9.8,6.0
2838951,20250514YORK1725,51747,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,17:25:00,...,0.0,2,NO,NO,YES,5.50,NO,-10.00,9.8,7.0
2838952,20250514YORK1725,51748,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,17:25:00,...,2.0,5,NO,YES,NO,6.80,YES,56.84,-58.0,9.0
2838953,20250514YORK1725,51749,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,17:25:00,...,0.0,3,NO,NO,NO,6.80,YES,56.84,-58.0,8.0


In [20]:
df["Place"] = pd.to_numeric(df["Place"], errors="coerce")
df["Place"] = df["Place"].fillna(df["Runners"])

df["normalized_position"] = (df["Place"].astype(int)) / (df["Runners"].astype(int))
df.loc[df["normalized_position"] > 1, "normalized_position"] = 1

df


,rid,Unnamed: 0,date,Country,Track,Going,Type,Distance,Class,Time,...,Last time out Position,Course Winner,Distance Winner,Up in Trip,Betfair Place SP,Placed in Betfair Market,Place Return,Place Lay Return,Betfair Rank,normalized_position
0,20020406AINTREE1345,0,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,...,,,,,NaN,,0.00,0.0,NaN,0.882353
1,20020406AINTREE1345,1,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,...,,,,,NaN,,0.00,0.0,NaN,0.294118
2,20020406AINTREE1345,2,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,...,,,,,NaN,,0.00,0.0,NaN,0.117647
3,20020406AINTREE1345,3,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,...,,,,,NaN,,0.00,0.0,NaN,0.941176
4,20020406AINTREE1345,4,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,...,,,,,NaN,,0.00,0.0,NaN,0.705882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838950,20250514YORK1725,51746,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,17:25:00,...,11,NO,YES,NO,3.97,NO,-10.00,9.8,6.0,0.714286
2838951,20250514YORK1725,51747,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,17:25:00,...,2,NO,NO,YES,5.50,NO,-10.00,9.8,7.0,0.500000
2838952,20250514YORK1725,51748,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,17:25:00,...,5,NO,YES,NO,6.80,YES,56.84,-58.0,9.0,0.214286
2838953,20250514YORK1725,51749,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,17:25:00,...,3,NO,NO,NO,6.80,YES,56.84,-58.0,8.0,0.142857


## Weight

In [21]:
def splitwt(wt, ind):
    try:
        wt = pd.to_numeric(wt.split("-")[0])
        return wt
    except:
        return pd.NA
    
df["Weight"]  = df["Weight"].str.replace("\xa0", "")

df["weightSt"] = df["Weight"].apply(splitwt, ind=0)
df["weightLb"] = df["Weight"].apply(splitwt, ind=1)
df["weight_total_lb"] = df["weightSt"] * 14 + df["weightLb"]

df["race_avg_weight"] = df.groupby("rid")["weight_total_lb"].transform("mean")
df["weight_diff_pct_from_race_avg"] = (df["weight_total_lb"] - df["race_avg_weight"])/df["race_avg_weight"]

df

,rid,Unnamed: 0,date,Country,Track,Going,Type,Distance,Class,Time,...,Placed in Betfair Market,Place Return,Place Lay Return,Betfair Rank,normalized_position,weightSt,weightLb,weight_total_lb,race_avg_weight,weight_diff_pct_from_race_avg
0,20020406AINTREE1345,0,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,...,,0.00,0.0,NaN,0.882353,11.0,11.0,165.0,151.764706,0.087209
1,20020406AINTREE1345,1,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,...,,0.00,0.0,NaN,0.294118,10.0,10.0,150.0,151.764706,-0.011628
2,20020406AINTREE1345,2,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,...,,0.00,0.0,NaN,0.117647,10.0,10.0,150.0,151.764706,-0.011628
3,20020406AINTREE1345,3,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,...,,0.00,0.0,NaN,0.941176,11.0,11.0,165.0,151.764706,0.087209
4,20020406AINTREE1345,4,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,...,,0.00,0.0,NaN,0.705882,10.0,10.0,150.0,151.764706,-0.011628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838950,20250514YORK1725,51746,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,17:25:00,...,NO,-10.00,9.8,6.0,0.714286,9.0,9.0,135.0,131.785714,0.024390
2838951,20250514YORK1725,51747,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,17:25:00,...,NO,-10.00,9.8,7.0,0.500000,8.0,8.0,120.0,131.785714,-0.089431
2838952,20250514YORK1725,51748,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,17:25:00,...,YES,56.84,-58.0,9.0,0.214286,8.0,8.0,120.0,131.785714,-0.089431
2838953,20250514YORK1725,51749,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,17:25:00,...,YES,56.84,-58.0,8.0,0.142857,9.0,9.0,135.0,131.785714,0.024390


## Distance

In [22]:
def convert_distance_to_furlongs(df, col="Distance"):
    """
    Converts a distance string like '2m', '1m6f', '2m1½f' into a float number of furlongs.
    1 mile = 8 furlongs, so '1m6f' = 14 furlongs.
    """
    import re

    def parse_distance(value):
        if pd.isna(value):
            return np.nan

        # Extract miles and furlongs (including fractional furlongs)
        miles = 0
        furlongs = 0.0

        # Match miles and furlongs
        mile_match = re.search(r"(\d+)m", value)
        if mile_match:
            miles = int(mile_match.group(1))

        # Match furlongs including fractions (e.g., ½)
        furlong_match = re.search(r"(\d+)?(?:½)?f", value)
        if furlong_match:
            f_str = furlong_match.group(1)
            if f_str:
                furlongs += int(f_str)
            if "½f" in value:
                furlongs += 0.5

        return miles * 8 + furlongs

    df["distance_furlongs"] = df[col].apply(parse_distance)
    return df

df = convert_distance_to_furlongs(df)

df['distance_bin'] = pd.qcut(df['distance_furlongs'], q=5)


df

,rid,Unnamed: 0,date,Country,Track,Going,Type,Distance,Class,Time,...,Place Lay Return,Betfair Rank,normalized_position,weightSt,weightLb,weight_total_lb,race_avg_weight,weight_diff_pct_from_race_avg,distance_furlongs,distance_bin
0,20020406AINTREE1345,0,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,...,0.0,NaN,0.882353,11.0,11.0,165.0,151.764706,0.087209,16.5,"(16.0, 19.5]"
1,20020406AINTREE1345,1,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,...,0.0,NaN,0.294118,10.0,10.0,150.0,151.764706,-0.011628,16.5,"(16.0, 19.5]"
2,20020406AINTREE1345,2,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,...,0.0,NaN,0.117647,10.0,10.0,150.0,151.764706,-0.011628,16.5,"(16.0, 19.5]"
3,20020406AINTREE1345,3,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,...,0.0,NaN,0.941176,11.0,11.0,165.0,151.764706,0.087209,16.5,"(16.0, 19.5]"
4,20020406AINTREE1345,4,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,13:45:00,...,0.0,NaN,0.705882,10.0,10.0,150.0,151.764706,-0.011628,16.5,"(16.0, 19.5]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838950,20250514YORK1725,51746,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,17:25:00,...,9.8,6.0,0.714286,9.0,9.0,135.0,131.785714,0.024390,12.0,"(8.5, 16.0]"
2838951,20250514YORK1725,51747,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,17:25:00,...,9.8,7.0,0.500000,8.0,8.0,120.0,131.785714,-0.089431,12.0,"(8.5, 16.0]"
2838952,20250514YORK1725,51748,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,17:25:00,...,-58.0,9.0,0.214286,8.0,8.0,120.0,131.785714,-0.089431,12.0,"(8.5, 16.0]"
2838953,20250514YORK1725,51749,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,17:25:00,...,-58.0,8.0,0.142857,9.0,9.0,135.0,131.785714,0.024390,12.0,"(8.5, 16.0]"


## Going and race type bins

In [23]:
going_bin_map = {
    'FST': 'fast',
    'FRM': 'fast',
    'HRD': 'fast',
    'STFA': 'fast',
    'STFI': 'fast',

    'GD': 'good',
    'GTF': 'good',
    'GTS': 'good',
    'STG': 'good',
    'GTY': 'good',
    'YTG': 'good',
    'YTF': 'good',

    'SFT': 'soft',
    'STY': 'soft',
    'YLD': 'soft',
    'YTS': 'soft',

    'HVY': 'heavy',
    'HTS': 'heavy',
    'YSH': 'heavy',

    'SLW': 'slow',
    'STSL': 'slow',
    'STSF': 'slow',
    'STHE': 'slow',

    'STD': 'standard'
}

race_type_bin_map = {
    'Maiden': 'maiden',
    'Novice Stakes': 'maiden',

    'Handicap Hurdle': 'handicap',
    'Other Handicap': 'handicap',
    'Handicap Chase': 'handicap',
    'Selling Handicap': 'handicap',
    'Novice Hcap Chase': 'handicap',
    'Novice Hcap Hurdle': 'handicap',
    'Nursery': 'handicap',

    'Selling Stakes': 'stakes',
    'Claiming Stakes': 'stakes',
    'Classified Stakes': 'stakes',
    'Conditions Stakes': 'stakes',

    'Novice Hurdle': 'novice',
    'Novice Chase': 'novice',

    'Group 1': 'group',
    'Group 2': 'group',
    'Group 3': 'group',

    'Listed': 'listed',

    'Other Hurdle': 'hurdle',
    'Selling Hurdle': 'hurdle',

    'Other Chase': 'chase',

    'NH Flat': 'nh_flat',
    'Hunters Chase': 'hunters',
    'Amateur': 'amateur',
    'Unclassified': 'unclassified'
}


df['going_bin'] = df['Going'].map(going_bin_map)
df['race_type_bin'] = df['Type'].map(race_type_bin_map)

df.to_csv("preprocessed_before_form.csv")



In [38]:
# df2 = pd.read_csv("preprocessed_before_form.csv")
# df2['going_bin'].unique()
# df["Going"]
# df
# df = df2.copy()
df

,Unnamed: 0,rid,Unnamed: 0.1,date,Country,Track,Going,Type,Distance,Class,...,normalized_position,weightSt,weightLb,weight_total_lb,race_avg_weight,weight_diff_pct_from_race_avg,distance_furlongs,distance_bin,going_bin,race_type_bin
0,0,20020406AINTREE1345,0,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,...,0.882353,11.0,11.0,165.0,151.764706,0.087209,16.5,"(16.0, 19.5]",good,handicap
1,1,20020406AINTREE1345,1,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,...,0.294118,10.0,10.0,150.0,151.764706,-0.011628,16.5,"(16.0, 19.5]",good,handicap
2,2,20020406AINTREE1345,2,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,...,0.117647,10.0,10.0,150.0,151.764706,-0.011628,16.5,"(16.0, 19.5]",good,handicap
3,3,20020406AINTREE1345,3,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,...,0.941176,11.0,11.0,165.0,151.764706,0.087209,16.5,"(16.0, 19.5]",good,handicap
4,4,20020406AINTREE1345,4,2002-04-06,GB,AINTREE,GD,Handicap Hurdle,2m½f,1.0,...,0.705882,10.0,10.0,150.0,151.764706,-0.011628,16.5,"(16.0, 19.5]",good,handicap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838950,2838950,20250514YORK1725,51746,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,...,0.714286,9.0,9.0,135.0,131.785714,0.024390,12.0,"(8.5, 16.0]",good,handicap
2838951,2838951,20250514YORK1725,51747,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,...,0.500000,8.0,8.0,120.0,131.785714,-0.089431,12.0,"(8.5, 16.0]",good,handicap
2838952,2838952,20250514YORK1725,51748,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,...,0.214286,8.0,8.0,120.0,131.785714,-0.089431,12.0,"(8.5, 16.0]",good,handicap
2838953,2838953,20250514YORK1725,51749,2025-05-14,GB,YORK,GTF,Other Handicap,1m4f,4.0,...,0.142857,9.0,9.0,135.0,131.785714,0.024390,12.0,"(8.5, 16.0]",good,handicap


# Jockey, horse (and trainer?) trends

maybe ignore for trainer justb use simple trends. but do it for distance bins


In [39]:
import numpy as np


def form_trend(x):
    x = x.dropna()
    if len(x) < 2:
        return np.nan
    t = np.arange(len(x))
    slope = np.polyfit(t, x.values, 1)[0]
    return slope

def weighted_momentum(x):
    x = x.dropna()
    if len(x) == 0:
        return np.nan
    weights = np.exp(-0.5 * np.arange(len(x))[::-1]) 
    return np.average(x, weights=weights)

def get_trends(df, target, form_cols, filter_col=None, value="all"):

    df = df.copy()

    if value == "all" and filter_col is None:
        mask = df["rid"] == df["rid"]
    else:
        try:
            mask = df[f"{filter_col}"] == value
            # df = df.loc[df[f"{filter_col}"] == value].copy()
        except:
            print("Provide both filtering column and value")
            return df, form_cols

    cols_original = df.columns

    df.loc[mask,f"{target}_{value}_form_avg_3"] = (
        df.loc[mask].groupby(target)["normalized_position"]
        .transform(lambda x: x.shift(1).rolling(3, min_periods=1).mean())
    ).fillna(0.5)

    df.loc[mask,f"{target}_{value}_form_trend_3"] = (
        df.loc[mask].groupby(target)["normalized_position"]
        .transform(lambda x: x.shift(1).rolling(3, min_periods=2).apply(form_trend, raw=False))
    ).fillna(0)

    df.loc[mask,f"{target}_{value}_form_volatility_3"] = (
        df.loc[mask].groupby(target)["normalized_position"]
        .transform(lambda x: x.shift(1).rolling(3, min_periods=1).std())
    ).fillna(0)

    df.loc[mask,f"{target}_{value}_win_rate_5"] = (
        df.loc[mask].groupby(target)["normalized_position"]
        .transform(lambda x: x.shift(1).rolling(5, min_periods=1).apply(lambda s: (s == 0).mean()))
    ).fillna(0)

    df.loc[mask,f"{target}_{value}_form_momentum"] = (
        df.loc[mask].groupby(target)["normalized_position"]
        .transform(lambda x: x.shift(1).rolling(5, min_periods=1).apply(weighted_momentum, raw=False))
    ).fillna(0)

    form_cols += [c for c in df.columns if c not in cols_original]

    return df, form_cols

form_targets = ["Horse", "Jockey", "Trainer"]
goings = df["going_bin"].unique()
race_types = df["race_type_bin"].unique()

form_cols = []

for t in form_targets:
    df, form_cols = get_trends(df, t, form_cols, filter_col=None, value="all")

    for going in goings:
        df, form_cols = get_trends(df, t, form_cols, filter_col="going_bin", value=going)
        df.to_csv(f"progress/preprocessed_after_form_midway_{going}_{t}.csv")

    for race_type in race_types:
        df, form_cols = get_trends(df, t, form_cols, filter_col="race_type_bin", value=race_type)
        df.to_csv(f"progress/preprocessed_after_form_midway_{race_type}_{t}.csv")


df.to_csv("preprocessed_after_form.csv")


In [49]:
form_cols = []

for c in df.columns:
    # print(c)

    if 'all' in c:
        form_cols.append(c)


for c in df.columns[-30:]:
    form_cols.append(c)

form_cols


['Stall',
 'Horse_all_form_avg_3',
 'Horse_all_form_trend_3',
 'Horse_all_form_volatility_3',
 'Horse_all_win_rate_5',
 'Horse_all_form_momentum',
 'Jockey_all_form_avg_3',
 'Jockey_all_form_trend_3',
 'Jockey_all_form_volatility_3',
 'Jockey_all_win_rate_5',
 'Jockey_all_form_momentum',
 'Trainer_all_form_avg_3',
 'Trainer_all_form_trend_3',
 'Trainer_all_form_volatility_3',
 'Trainer_all_win_rate_5',
 'Trainer_all_form_momentum',
 'Horse_this_going_form_avg_3',
 'Horse_this_going_form_trend_3',
 'Horse_this_going_form_volatility_3',
 'Horse_this_going_win_rate_5',
 'Horse_this_going_form_momentum',
 'Horse_this_race_type_form_avg_3',
 'Horse_this_race_type_form_trend_3',
 'Horse_this_race_type_form_volatility_3',
 'Horse_this_race_type_win_rate_5',
 'Horse_this_race_type_form_momentum',
 'Jockey_this_going_form_avg_3',
 'Jockey_this_going_form_trend_3',
 'Jockey_this_going_form_volatility_3',
 'Jockey_this_going_win_rate_5',
 'Jockey_this_going_form_momentum',
 'Jockey_this_race_type

In [44]:
form_targets = ["Horse", "Jockey", "Trainer"]
goings = df["going_bin"].unique()
race_types = df["race_type_bin"].unique()




for target in form_targets:

    race_relevant_going_form_cols = [
        f"{target}_this_going_form_avg_3",
        f"{target}_this_going_form_trend_3",
        f"{target}_this_going_form_volatility_3",
        f"{target}_this_going_win_rate_5",
        f"{target}_this_going_form_momentum",
    ]

    for r in race_relevant_going_form_cols:
       df[r] = None

    for going in goings:
      mask = df["going_bin"] == going
      df.loc[mask, f"{target}_this_going_form_avg_3"] = df[f"{target}_{going}_form_avg_3"]
      df.loc[mask, f"{target}_this_going_form_trend_3"] = df[f"{target}_{going}_form_trend_3"]
      df.loc[mask, f"{target}_this_going_form_volatility_3"] = df[f"{target}_{going}_form_volatility_3"]
      df.loc[mask, f"{target}_this_going_win_rate_5"] = df[f"{target}_{going}_win_rate_5"]
      df.loc[mask, f"{target}_this_going_form_momentum"] = df[f"{target}_{going}_form_momentum"]

    race_relevant_race_type_form_cols = [
        f"{target}_this_race_type_form_avg_3",
        f"{target}_this_race_type_form_trend_3",
        f"{target}_this_race_type_form_volatility_3",
        f"{target}_this_race_type_win_rate_5",
        f"{target}_this_race_type_form_momentum",
    ]

    for r in race_relevant_race_type_form_cols:
       df[r] = None

    for race_type in race_types:
      mask = df["going_bin"] == going
      df.loc[mask, f"{target}_this_race_type_form_avg_3"] = df[f"{target}_{race_type}_form_avg_3"]
      df.loc[mask, f"{target}_this_race_type_form_trend_3"] = df[f"{target}_{race_type}_form_trend_3"]
      df.loc[mask, f"{target}_this_race_type_form_volatility_3"] = df[f"{target}_{race_type}_form_volatility_3"]
      df.loc[mask, f"{target}_this_race_type_win_rate_5"] = df[f"{target}_{race_type}_win_rate_5"]
      df.loc[mask, f"{target}_this_race_type_form_momentum"] = df[f"{target}_{race_type}_form_momentum"]
      



# Train and test sets

In [50]:
X_cols = [
    # "Horse",
    "rid",
    "going_bin",
    "race_type_bin",
    "distance_bin",
    "Official Rating",
    "Pace",
    "weight_total_lb",
    "weight_diff_pct_from_race_avg",
    "Age",
    "Class",
    "Headgear",
]

X_cols += form_cols

categorical_cols = [
        "going_bin",
        "race_type_bin",
        'distance_bin',
        "Age",
        "Class",
        "Headgear",
    ]

df["runners_actual"] = df.groupby("rid")["Horse"].transform("nunique")
df.loc[df["runners_actual"] != df["Runners"], "Runners"] = df["runners_actual"]

df["position_target"] = df["Runners"] - df["Place"]
df.loc[df["position_target"] > 20, "position_target"] = 20
y_col = "position_target"

df = df.dropna(subset=[y_col])

In [51]:
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split

# Change to scale/encode train data first

def prepare_training_data(df, X_cols, y_col=y_col):

    df = df.copy()


    # Select features and target
    X = df[X_cols]
    y = df[y_col]
    # df_val = df[val_cols]

    # Split by race and time
    rids_sorted = df.sort_values("date")["rid"].drop_duplicates().values
    n = len(rids_sorted)

    # 70% train, 10% val, 20% test
    train_rids = rids_sorted[:int(0.7 * n)]
    # val_rids   = rids_sorted[int(0.7 * n):int(0.8 * n)]
    test_rids  = rids_sorted[int(0.7 * n):]


    # Separate features into types
    categorical_cols = [
        "Going",
        "Type",
        "Age",
        "Class",
        "Headgear",
    ]

    # Fill missing values
    X[categorical_cols] = X[categorical_cols].fillna("Unknown")
    for c in categorical_cols:
        X[f"{c}_unknown"] = 0
        X.loc[X[c] == "Unknown", f"{c}_unknown"] = 1

    X[categorical_cols] = X[categorical_cols].astype("category")

     # Apply to X and y
    X_train = X[df["rid"].isin(train_rids)]
    y_train = y[df["rid"].isin(train_rids)]

    # df_val = df_val[df["rid"].isin(val_rids)]

    X_test = X[df["rid"].isin(test_rids)]
    y_test = y[df["rid"].isin(test_rids)]

    # Return all sets
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = prepare_training_data(df, X_cols)

group_train = X_train.groupby('rid').size().to_numpy()
group_test = X_test.groupby('rid').size().to_numpy()

X_train = X_train.drop("rid", axis=1)
X_test = X_test.drop("rid", axis=1)

X_train

: 

In [ ]:
# Deal with properly later
y_train.loc[y_train <0] = 0
y_test.loc[y_test <0] = 0


In [ ]:

train_data = lgb.Dataset(X_train, label=y_train, group=group_train, categorical_feature=categorical_cols)
test_data = lgb.Dataset(X_test, label=y_test, group=group_test, reference=train_data, categorical_feature=categorical_cols)

# Train a LambdaRank model
params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'ndcg_eval_at': [1, 3],
    'learning_rate': 0.1,
    'num_leaves': 31
}

model = lgb.train(params, train_data, valid_sets=[test_data], num_boost_round=100)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.076129 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2577
[LightGBM] [Info] Number of data points in the train set: 2031437, number of used features: 21
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines


In [ ]:
from sklearn.metrics import ndcg_score
import numpy as np
from tqdm import tqdm

y_pred = model.predict(X_test)

ndcg_scores = []
start = 0
for g in tqdm(group_test):
    end = start + g
    
    true_relevance = y_test.iloc[start:end].to_numpy().reshape(1, -1)
    predicted_scores = y_pred[start:end].reshape(1, -1)


    try:
        ndcg = ndcg_score(true_relevance, predicted_scores, k=3)
        ndcg_scores.append(ndcg)
    except:
        # print(true_relevance)
        # print(predicted_scores)
        pass

    start = end

print("Average NDCG@3:", np.mean(ndcg_scores))



100%|██████████| 84640/84640 [00:39<00:00, 2168.56it/s]

Average NDCG@3: 0.6925300084140382


In [ ]:
eval_df = df.loc[X_test.index]
eval_df["pred_score"] = y_pred
eval_df['pred_rank'] = eval_df.groupby('rid')['pred_score'].rank(ascending=False, method='first')
eval_df["pred_win"] = eval_df["pred_rank"] == 1

races_bet_on = eval_df["rid"].nunique()
stake = races_bet_on * 10

winnings_betfair = eval_df.loc[(eval_df["pred_win"])&(eval_df["Place"] == 1),"Betfair Win Return"].sum()
winnings_sp = eval_df.loc[(eval_df["pred_win"])&(eval_df["Place"] == 1),"SP Win Return"].sum()
# winnings_ew = eval_df.loc[(eval_df["pred_win"])&(eval_df["Place"] == 1),"E/W Return"].sum()
# winnings_fc = eval_df.loc[(eval_df["pred_win"])&(eval_df["Place"] == 1),"Forecasted Odds"].sum()*10

profit_betfair = (winnings_betfair-stake)/stake
profit_sp = (winnings_sp-stake)/stake
# profit_ew = (winnings_ew-stake)/stake
# profit_fc = (winnings_fc-stake)/stake

# print(profit_betfair, profit_sp, profit_ew, profit_fc)
print(profit_betfair, profit_sp)

races_won = eval_df.loc[(eval_df["pred_win"])&(eval_df["Place"] == 1)].shape[0]
print(f"No of races: {races_bet_on}")
print(f"No of races won: {races_won}")
print(f"Win pred ratio: {races_won/races_bet_on}")


-0.27600419423440464 -0.38753544423440456
No of races: 84640
No of races won: 21103
Win pred ratio: 0.2493265595463138
